In [50]:
# Imports
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split

print('CUDA support') if torch.cuda.is_available() else print('No CUDA support') 

No CUDA support


### 1. Dataloader
PyTorch makes it a bit harder to load in custom data.  
I will read the data with pandas

In [51]:
# Read csv file
math_df = pd.read_csv("../Data/Math_Test_Results_Cleaned.csv")

# X -> features  
# y -> labels
X = math_df.drop('Mean Scale Score', axis=1)
y = math_df['Mean Scale Score']

Next create a custom dataset  
This just converts the X and y values to torch tensors and wraps it in a class

In [52]:
class MathDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values).to(torch.float32)
        self.y = torch.tensor(y).to(torch.float32)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx])

Use the class to process the data

In [53]:
torch_dataset = MathDataset(X, y)
print("length of the dataset is:", len(torch_dataset))

length of the dataset is: 25391


Split the data into train and test data, the pytorch way

In [54]:
train_data, test_data = random_split(torch_dataset, [20000, 5391])

print("The length of train data is:",len(train_data))
print("The length of test data is:",len(test_data))

The length of train data is: 20000
The length of test data is: 5391


In [55]:
print(train_data[0])

(tensor([3.0000e+00, 2.0090e+03, 6.2000e+01, 2.0000e+00, 3.2000e+00, 8.0000e+00,
        1.2900e+01, 3.5000e+01, 5.6500e+01, 1.7000e+01, 2.7400e+01, 2.7000e+01,
        3.0000e+00, 1.8300e+02]), tensor(684.))


This data is not normalized yet.  
I will do this with the sklearn StandardScaler.  

In [56]:
scaler = StandardScaler()
train_data.dataset.X = scaler.fit_transform(train_data.dataset.X)
test_data.dataset.X = scaler.transform(test_data.dataset.X)

In [57]:
print(train_data[0])

(array([-3.90454249e+00, -1.00857901e+03, -1.22211792e+00, -7.12020905e-01,
       -7.98038280e-01, -9.27517182e-01, -1.62985082e+00, -1.13688535e+00,
       -2.57980308e+00, -7.40990302e-01, -1.08154663e+00, -1.23124724e+00,
       -5.59778364e-01, -1.21892869e+00]), tensor(684.))


There we go!  
Next I will need mini-batches  
This is done with the DataLoader class to make things simple

In [99]:
train_loader = DataLoader(dataset=train_data, batch_size=50, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=20, shuffle=True)

Now we have our data ready for use.  

### 2. First modelling
In Pytorch, this is also done in a seperate class  
This one inherits from the Module class  
I will start with a very simple linear model to get the pipeline started

In [132]:
class LinearModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = torch.nn.Linear(14, 1)
    
    def forward(self, x):
        x = self.layer(x)
        return x

We also need a class that trains our model:

In [158]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(y, yhat)
        loss.backward()
        optimizer.step() # update parameters
        optimizer.zero_grad()
        return loss.item()
    return train_step

And this is a simple implementation

In [162]:
model = LinearModel()
loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
train_step = make_train_step(model, loss_fn, optimizer)

for x_batch, y_batch in train_loader:
    loss = train_step(x_batch.to(torch.float32), y_batch)
    print(loss)

720959.75
2979760701440.0
1.2322141050578665e+19
5.094668817474492e+25
2.1064907633921337e+32
inf
inf
inf
inf
inf
inf
inf
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
na

Mmh... this is an exploding gradient.